In [10]:
# Environment and Dependencies:
# - Python 3.8.10
# - pandas 1.4.2
# - numpy 1.22.3
# - openpyxl 3.0.9
# - zipfile (built-in)

import pandas as pd
import zipfile
from io import BytesIO
from openpyxl import load_workbook

# 1. Define the list of periods and expand to individual years
periods = ['2002-2005', '2006-2009', '2010-2013']
years = []
for pr in periods:
    start, end = map(int, pr.split('-'))
    years.extend(range(start, end + 1))
years = sorted(set(years))
year_strs = [str(y) for y in years]

# 2. Read the WDI Excel file from within the ZIP archive into memory
zip_path = 'WDI_excel_2016_06.zip'
with zipfile.ZipFile(zip_path) as zf:
    # Find the first .xls or .xlsx file in the archive
    excel_name = next(name for name in zf.namelist() if name.lower().endswith(('.xls', '.xlsx')))
    binary_excel = zf.read(excel_name)

# 3. Auto-detect the header row in the 'Data' sheet using openpyxl
wb = load_workbook(filename=BytesIO(binary_excel), read_only=True)
ws = wb['Data']
header_row = None
for idx, row in enumerate(ws.iter_rows(min_row=1, max_row=20, values_only=True), start=1):
    if row[:4] == ('Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'):
        header_row = idx
        break
if header_row is None:
    raise ValueError('Header row with expected columns not found')

# 4. Load the 'Data' sheet into a pandas DataFrame using the detected header
wdi_df = pd.read_excel(
    BytesIO(binary_excel),
    sheet_name='Data',
    header=header_row-1,
    engine='openpyxl'
)
wdi_df.columns = wdi_df.columns.str.strip()  # strip whitespace

# 5. Identify and keep only necessary columns: base columns + target years
base_cols = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']
required_cols = base_cols + year_strs
available_cols = [col for col in required_cols if col in wdi_df.columns]
missing = set(required_cols) - set(available_cols)
if missing:
    print(f"Warning: Missing and skipped columns: {sorted(missing)}")
wdi_subset = wdi_df[available_cols].copy()

# 6. Filter for the eight agricultural indicators of interest
indicator_map = {
    'TM.VAL.AGRI.ZS.UN': 'Agri_Imp_pct',
    'TX.VAL.AGRI.ZS.UN': 'Agri_Exp_pct',
    'AG.PRD.CROP.XD':    'Crop_Index',
    'AG.PRD.FOOD.XD':    'Food_Index',
    'AG.PRD.LVSK.XD':    'Livestock_Index',
    'NV.AGR.TOTL.ZS':    'Agri_ValueAdded_pctGDP',
    'NV.AGR.TOTL.KD.ZG': 'Agri_ValueAdded_Growth',
    'EA.PRD.AGRI.KD':    'Agri_ValueAdded_perCapita'
}
wdi_filtered = wdi_subset[wdi_subset['Indicator Code'].isin(indicator_map.keys())]

# 7. Melt from wide to long format
wdi_long = wdi_filtered.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Code'],
    value_vars=[yr for yr in year_strs if yr in wdi_filtered.columns],
    var_name='Year',
    value_name='Value'
)
wdi_long['Year'] = wdi_long['Year'].astype(int)

# 8. Pivot back to wide format and rename indicator columns
wdi_wide = (
    wdi_long
    .pivot(index=['Country Name', 'Year'], columns='Indicator Code', values='Value')
    .reset_index()
    .rename(columns=indicator_map)
)

# 9. Map each Year back to its period and compute mean values per country-period

def assign_period(y):
    if 2002 <= y <= 2005:
        return '2002-2005'
    if 2006 <= y <= 2009:
        return '2006-2009'
    if 2010 <= y <= 2013:
        return '2010-2013'
    return None

wdi_wide['Period'] = wdi_wide['Year'].apply(assign_period)
summary_df = (
    wdi_wide
    .groupby(['Country Name', 'Period'], as_index=False)
    .mean()
    .rename(columns={'Country Name': 'Country'})
)

# 10. Save the summarized output to CSV without relying on an external CSV input
summary_df.to_csv('R6_extracted_from_WDI.csv', index=False)
print('Extraction complete: R6_extracted_from_WDI.csv saved')


Done! 文件已保存：R6_extracted_from_WDI.csv
Comparison file saved to R6_comparison.csv
